In [1]:
# import pandas as pd

# # List of CSV files to merge
# csv_files = ['dares/test.csv', 'dares/train.csv', 'dares/validation.csv']

# # Read and concatenate all CSV files
# merged_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

# # Save the merged dataframe to a new CSV file
# merged_df.to_csv('merged_output.csv', index=False)

# print("CSV files merged successfully into 'merged_output.csv'")

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("merged_output.csv",sep="\t")

In [4]:
df_math = df[df["English_Filename"]=="Math"].reset_index(drop=True)
df_Science = df[df["English_Filename"]=="Science"].reset_index(drop=True)
df_Physics = df[df["English_Filename"]=="Physics"].reset_index(drop=True)
df_Ecology = df[df["English_Filename"]=="Ecology"].reset_index(drop=True)
df_Chemistry = df[df["English_Filename"]=="Chemistry"].reset_index(drop=True)
df_Biology = df[df["English_Filename"]=="Biology"].reset_index(drop=True)

In [5]:
df_Arabic1 = df[df["English_Filename"]=="ArabicLanguage"].reset_index(drop=True)
df_Arabic2 = df[df["English_Filename"]=="Arabic_Language"].reset_index(drop=True)

In [6]:
df_Arabic = pd.concat([df_Arabic1,df_Arabic2], ignore_index=True)

In [7]:
# import faiss
# import numpy as np
# from FlagEmbedding import BGEM3FlagModel
# import torch
# from tqdm.notebook import tqdm
# import pickle

# # Ensure GPU is used if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if device.type != "cuda":
#     raise RuntimeError("CUDA device not available. Ensure GPU and faiss-gpu are properly installed.")

# # Initialize the BGE-M3 model
# model = BGEM3FlagModel('BAAI/bge-m3', device=device)

# # Initialize FAISS GPU resources and index
# dimension = 1024  # BGE-M3 embedding dimension
# res = faiss.StandardGpuResources()  # Create GPU resources
# index = faiss.IndexFlatL2(dimension)  # Create CPU index
# gpu_index = faiss.index_cpu_to_gpu(res, 0, index)  # Move index to GPU (device 0)

# # List to store metadata
# metadata = []

# # Function to process and add embeddings for a given dataframe
# def add_embeddings_to_index(df, subject_name, index, metadata_list):
#     embeddings = []
#     for i in tqdm(range(len(df)), desc=f"Encoding {subject_name}"):
#         emb = model.encode(df["Text"][i], max_length=256)['dense_vecs']
#         embeddings.append(emb)
        
#         # Store only text and subject as metadata
#         meta = {'subject': subject_name, 'text': df["Text"][i]}
#         metadata_list.append(meta)
    
#     # Convert embeddings to numpy array and add to FAISS GPU index
#     embeddings = np.array(embeddings).astype('float32')
#     index.add(embeddings)
#     return embeddings

# # Process each subject's dataframe
# subjects = [
#     ('Math', df_math),
#     ('Arabic', df_Arabic),
#     ('Science', df_Science),
#     ('Physics', df_Physics),
#     ('Ecology', df_Ecology),
#     ('Chemistry', df_Chemistry),
#     ('Biology', df_Biology)
# ]

# for subject_name, df in subjects:
#     add_embeddings_to_index(df, subject_name, gpu_index, metadata)

# # Move index back to CPU for saving (optional, if you need CPU compatibility)
# cpu_index = faiss.index_gpu_to_cpu(gpu_index)
# faiss.write_index(cpu_index, "subject_embeddings.faiss")

# # Save metadata
# with open("subject_metadata.pkl", "wb") as f:
#     pickle.dump(metadata, f)

# print(f"FAISS GPU index created with {gpu_index.ntotal} vectors.")

In [2]:
import faiss
import numpy as np
from FlagEmbedding import BGEM3FlagModel
import torch
import pickle
from tqdm import tqdm

# Ensure GPU is used if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type != "cuda":
    raise RuntimeError("CUDA device not available. Ensure GPU and faiss-gpu are properly installed.")

# Initialize the BGE-M3 model
model = BGEM3FlagModel('BAAI/bge-m3', device=device)

# Load the FAISS index
cpu_index = faiss.read_index("subject_embeddings.faiss")

# Move index to GPU
res = faiss.StandardGpuResources()
gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)

# Load metadata
with open("subject_metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

# Function to query the FAISS index with subject filter
def search_faiss_with_subject_filter(query_text, index, model, metadata, subject_filter=None, top_k=5):
    # Encode the query text
    query_embedding = model.encode(query_text, max_length=256)['dense_vecs']
    query_embedding = np.array([query_embedding]).astype('float32')  # Shape: (1, dimension)

    # Perform similarity search (retrieve more results to account for filtering)
    search_k = top_k * 10  # Oversample to ensure enough results after filtering
    distances, indices = index.search(query_embedding, search_k)

    # Filter results by subject and collect top-k
    results = []
    for i, idx in enumerate(indices[0]):
        if idx >= len(metadata):  # Safety check for index bounds
            continue
        meta = metadata[idx]
        # Apply subject filter (case-insensitive)
        if subject_filter is None or meta['subject'].lower() == subject_filter.lower():
            result = {
                'index': int(idx),
                'distance': float(distances[0][i]),
                'subject': meta['subject'],
                'text': meta['text']
            }
            results.append(result)
        if len(results) >= top_k:  # Stop once we have enough filtered results
            break
    
    return results

# Example query with subject filter
query = "ما هو الفعل"
subject_filter = None  # Set to None for no filtering, or specify a subject like "Math"
top_k = 5  # Number of top results to retrieve
results = search_faiss_with_subject_filter(query, gpu_index, model, metadata, subject_filter, top_k)

# Print results
print(f"Query: {query}")
print(f"Subject Filter: {subject_filter if subject_filter else 'None'}")
print(f"Top {len(results)} results (up to {top_k}):")
for i, result in enumerate(results, 1):
    print(f"\nResult {i}:")
    print(f"Subject: {result['subject']}")
    print(f"Text: {result['text']}")
    print(f"Distance: {result['distance']:.4f}")

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Query: ما هو الفعل
Subject Filter: None
Top 5 results (up to 5):

Result 1:
Subject: Arabic
Text: الجملة الفعلية تتألف من ركنين أساسيين الفعل والفاعل. الفاعل هو اسم مرفوع يدل على الشخص القائم بالفعل. علامة رفع الفاعل هي الضمة التي تظهر على الحرف الأخير. أضع الأسماء التالية في جملة تكون فيها فاعلا، ثم أرددها بنطق سليم. أضع فاعلا لكل فعل من الأفعال التالية ثم أرددها بتلاوة سليمة. اقرأ الفقرة التالية، وعرف الفعل والفاعل، ونطقهما نطقا سليما شرقت شمس العيد فبادرت السعادة بالجميع، وظهرت مظاهر العيد على كل الوجوه، وفي كل البيوت والشوارع تبتسم الأفواه وتردد الألسن العبارات التهنئة. يرتدي الصغير والكبير الملابس الجديدة، ويحصل الأطفال على العيديات. أجب عن الأسئلة التالية بجملة فعلية واقرأها بالضبط الصحيح ماذا فعل التلميذ في الدرس؟ أجعل الفاعل مفردا في الجمل التالية وضبطها بالشكل طاف الحاج حول الكعبة. أجعل الفاعل جمعا في الجمل التالية وضبطها بالشكل انتصر الجنود على الأعداء. يلعب الأولاد بالكرة.
Distance: 0.7882

Result 2:
Subject: Arabic
Text: واستبدل الكلمات غير المستعملة أو غير المفهومة أو المك

In [ ]:
import unsloth
from unsloth import FastLanguageModel  # Import Unsloth
import faiss
import numpy as np
from FlagEmbedding import BGEM3FlagModel
import torch
from transformers import AutoTokenizer
from typing import List, Dict
import pickle
import os

# Ensure GPU is used if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type != "cuda":
    print("Warning: CUDA not available. Using CPU, which may be slower.")

# Initialize the BGE-M3 model for embeddings
embedding_model = BGEM3FlagModel('BAAI/bge-m3', device=device)

# Load the FAISS index
cpu_index = faiss.read_index('subject_embeddings.faiss')

# Move index to GPU if available
if device.type == "cuda":
    res = faiss.StandardGpuResources()
    gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)
else:
    gpu_index = cpu_index  # Fallback to CPU index

# Load metadata
metadata_path = 'subject_metadata.pkl'
with open(metadata_path, "rb") as f:
    metadata = pickle.load(f)

# Initialize the Qwen2.5-3B-Instruct model and tokenizer with Unsloth
llm_model_name = "Qwen/Qwen2.5-3B-Instruct"
llm_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=llm_model_name,
    max_seq_length=2048, # Use 4-bit quantization for memory savings
    device_map="auto"     # Automatically map to available devices
)

# Function to query FAISS without subject filter and no oversampling
def search_faiss(query_text: str, index, embedding_model, metadata: List[Dict], top_k: int = 3) -> List[Dict]:
    query_embedding = embedding_model.encode(query_text, max_length=256)['dense_vecs']
    query_embedding = np.array([query_embedding]).astype('float32')

    search_k = top_k
    distances, indices = index.search(query_embedding, search_k)

    results = []
    for i, idx in enumerate(indices[0]):
        if idx >= len(metadata):
            continue
        result = {
            'index': int(idx),
            'distance': float(distances[0][i]),
            'subject': metadata[idx]['subject'],
            'text': metadata[idx]['text']
        }
        results.append(result)
        if len(results) >= top_k:
            break
    
    return results

# Function to call the Qwen2.5-3B-Instruct model with chat template
def call_llm(prompt: str, max_new_tokens: int = 512) -> str:
    # Apply chat template with enable_thinking=False
    text = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(llm_model.device)

    # Generate response
    generated_ids = llm_model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.3,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    # Decode the response
    content = tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")
    return content

# Conversational RAG function
def conversational_rag(query: str) -> Dict[str, str]:
    results = search_faiss(query, gpu_index, embedding_model, metadata, top_k=3)
    context = "\n".join([f"- {result['text']} (موضوع: {result['subject']})" for result in results])
    
    prompt= f"""انت معلم ذكي وودود، وظيفتك مساعدة الطلاب من جميع المراحل الدراسية العمرية عن طريق الرد على أسئلتهم في مواضيع الرياضيات، الأحياء، الكيمياء، الفيزياء، البيئة، اللغة العربية، والعلوم بطريقة بسيطة وسهلة وواضحة تتناسب مع مستواهم العمري وفقًا للسياق المقدم. يجب استخدام اللغة العربية فقط في الإجابة على أسئلة الطلاب، مع مراعاة تقديم الإجابات بشكل دقيق ومناسب للسياق.

# تعليمات إضافية
* تأكد من أن الإجابة خالية من التعقيدات ومباشرة.
* إذا كان السؤال يتطلب شرحًا علميًا، استخدم لغة مبسطة تناسب المرحلة العمرية للطالب.
* لا تستخدم مصطلحات معقدة إلا إذا كانت ضرورية، وفي هذه الحالة اشرحها ببساطة.

السؤال: 
{query}

السياق:
{context}
"""
    
    response = call_llm(prompt)
    return {"content": response}

# Example conversational loop
def main():
    print("مرحبًا يا أصدقاء! أنا معلم ودود هنا لمساعدتكم في الدراسة من الصف الأول إلى الصف الثاني عشر. اسألوني سؤالًا بالعربية (اكتب 'خروج' للخروج):")
    while True:
        query = input("السؤال: ")
        if query.strip() == "خروج":
            break
        result = conversational_rag(query)
        print(f"الإجابة: {result['content']}\n")

if __name__ == "__main__":
    main()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.12: Fast Qwen2 patching. Transformers: 4.53.1.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.625 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
مرحبًا يا أصدقاء! أنا معلم ودود هنا لمساعدتكم في الدراسة من الصف الأول إلى الصف الثاني عشر. اسألوني سؤالًا بالعربية (اكتب 'خروج' للخروج):


السؤال:  ما هو الفعل في الجمله الفعليه


الإجابة: بالطبع، سأساعدك في هذا السؤال. في الجملة "قالت زهرة كلمة حلوة" الفعل هنا هو "قالت" وهو فعل مضارع مبني على الفتح، والفاعل هنا هو "زهرة". 

في الجملة "أعطى أحمد كتابه لأمها" الفعل هنا هو "أعطى" وهو فعل ماضٍ مبني على الفتح، والفاعل هنا هو "أحمد".

في الجملة "تذاخر الأمانات في القلوب" الفعل هنا هو "تذاخر" وهو فعل ماضٍ مبني على الفتح، والفاعل هنا هو "الأمانات".

في الجملة "تسللت النجوم إلى السماء" الفعل هنا هو "تسللت" وهو فعل ماضٍ مبني على الفتح، والفاعل هنا هو "النجوم".

في الجملة "أقبلت الشمس على الأرض" الفعل هنا هو "أقبلت" وهو فعل ماضٍ مبني على الفتح، والفاعل هنا هو "الشمس".

في الجملة "تجمعت الحشرات حول الطعام" الفعل هنا هو "تجمعت" وهو فعل ماضٍ مبني على الفتح، والفاعل هنا هو "الحشرات".

في الجملة "تعددت الأرقام في الحساب" الفعل هنا هو "تعددت" وهو فعل ماضٍ مبني على الفتح، والفاعل هنا هو "الأرقام".

في الجملة "تبلورت الرغبات في الأحلام" الفعل هنا هو "تبلورت" وهو فعل ماضٍ مبني على الفتح، والفاعل هنا هو "الرغبات".

في الجملة "تكتسبت الثقة في النفس" الفعل هنا هو "تكتسبت" وهو فعل ماض

In [ ]:
import unsloth
from unsloth import FastLanguageModel
import faiss
import numpy as np
from FlagEmbedding import BGEM3FlagModel
import torch
from transformers import AutoTokenizer
from typing import List, Dict
import pickle
import os
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

# Ensure GPU is used if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type != "cuda":
    print("Warning: CUDA not available. Using CPU, which may be slower.")

# Initialize the BGE-M3 model for embeddings
embedding_model = BGEM3FlagModel('BAAI/bge-m3', device=device)

# Load the FAISS index
cpu_index = faiss.read_index('subject_embeddings.faiss')

# Move index to GPU if available
if device.type == "cuda":
    res = faiss.StandardGpuResources()
    gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)
else:
    gpu_index = cpu_index  # Fallback to CPU index

# Load metadata
metadata_path = 'subject_metadata.pkl'
with open(metadata_path, "rb") as f:
    metadata = pickle.load(f)

# Initialize the Qwen2.5-3B-Instruct model and tokenizer with Unsloth
llm_model_name = "Qwen/Qwen2.5-3B-Instruct"
llm_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=llm_model_name,
    max_seq_length=2048,
    device_map="auto"
)

# Initialize LangChain memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="query",
    output_key="content",
    return_messages=True
)

# Define the prompt template with memory
prompt_template = PromptTemplate(
    input_variables=["chat_history", "query", "context"],
    template="""انت معلم ذكي وودود، وظيفتك مساعدة الطلاب من جميع المراحل الدراسية العمرية عن طريق الرد على أسئلتهم في مواضيع الرياضيات، الأحياء، الكيمياء، الفيزياء، البيئة، اللغة العربية، والعلوم بطريقة بسيطة وسهلة وواضحة تتناسب مع مستواهم العمري وفقًا للسياق المقدم. يجب استخدام اللغة العربية فقط في الإجابة على أسئلة الطلاب، مع مراعاة تقديم الإجابات بشكل دقيق ومناسب للسياق.

# تعليمات إضافية
* تأكد من أن الإجابة خالية من التعقيدات ومباشرة.
* إذا كان السؤال يتطلب شرحًا علميًا، استخدم لغة مبسطة تناسب المرحلة العمرية للطالب.
* لا تستخدم مصطلحات معقدة إلا إذا كانت ضرورية، وفي هذه الحالة اشرحها ببساطة.
* إذا كان هناك سجل محادثة سابق، استخدمه لضمان استمرارية السياق.

سجل المحادثة السابقة:
{chat_history}

السؤال: 
{query}

السياق:
{context}"""
)

# Function to query FAISS without subject filter and no oversampling
def search_faiss(query_text: str, index, embedding_model, metadata: List[Dict], top_k: int = 3) -> List[Dict]:
    query_embedding = embedding_model.encode(query_text, max_length=256)['dense_vecs']
    query_embedding = np.array([query_embedding]).astype('float32')

    search_k = top_k
    distances, indices = index.search(query_embedding, search_k)

    results = []
    for i, idx in enumerate(indices[0]):
        if idx >= len(metadata):
            continue
        result = {
            'index': int(idx),
            'distance': float(distances[0][i]),
            'subject': metadata[idx]['subject'],
            'text': metadata[idx]['text']
        }
        results.append(result)
        if len(results) >= top_k:
            break
    
    return results

# Function to call the Qwen25-3B-Instruct model with chat template
def call_llm(prompt: str, max_new_tokens: int = 512) -> str:
    text = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(llm_model.device)

    # Generate response
    generated_ids = llm_model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.3,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    # Decode the response
    content = tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")
    return content

# Conversational RAG function with memory
def conversational_rag(query: str) -> Dict[str, str]:
    # Search FAISS for relevant context
    results = search_faiss(query, gpu_index, embedding_model, metadata, top_k=3)
    context = "\n".join([f"- {result['text']} (موضوع: {result['subject']})" for result in results])

    # Get chat history from memory
    chat_history = memory.load_memory_variables({})["chat_history"]
    
    # Format the prompt with history, query, and context
    prompt = prompt_template.format(
        chat_history=chat_history,
        query=query,
        context=context
    )

    # Generate response
    response = call_llm(prompt)

    # Save the query and response to memory
    memory.save_context({"query": query}, {"content": response})

    return {"content": response}

# Example conversational loop
def main():
    print("مرحبًا يا أصدقاء! أنا معلم ودود هنا لمساعدتكم في الدراسة من الصف الأول إلى الصف الثاني عشر. اسألوني سؤالًا بالعربية (اكتب 'خروج' للخروج):")
    while True:
        query = input("السؤال: ")
        if query.strip() == "خروج":
            break
        result = conversational_rag(query)
        print(f"الإجابة: {result['content']}\n")

if __name__ == "__main__":
    main()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.12: Fast Qwen2 patching. Transformers: 4.53.1.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.625 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/tmp/ipykernel_16791/3765256397.py:46: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


مرحبًا يا أصدقاء! أنا معلم ودود هنا لمساعدتكم في الدراسة من الصف الأول إلى الصف الثاني عشر. اسألوني سؤالًا بالعربية (اكتب 'خروج' للخروج):


السؤال:  ما هو الفعل في الجمله الفعليه


الإجابة: حسنًا، سأبدأ بالأسئلة التي قدمتها. سأقوم بتوضيح الفعل والفاعل في الجمل الفعلية المعطاة لك، ثم أقرأ الجمل بنطق صحيح.

1. "قال أبو محمد الحمدان"
   - الفعل: قال
   - الفاعل: أبو محمد الحمدان

2. "أتى الخادم إلى المنزل"
   - الفعل: أتى
   - الفاعل: الخادم

3. "انطفأت النور"
   - الفعل: انطفأت
   - الفاعل: النور

4. "أحضرت الأم الطعام"
   - الفعل: حضرت
   - الفاعل: الأم

5. "استيقظت السيدة على الناقص"
   - الفعل: استيقظت
   - الفاعل: السيدة

أرجو أن يكون هذا مفيدًا لك! إذا كنت تحتاج إلى توضيح أكثر أو تريد مزيدًا من الأمثلة، فلا تتردد في سؤالي.



السؤال:  ماذا بعد الفعل


الإجابة: حسنًا، سأبدأ بالأسئلة التي قدمتها. سأقوم بتوضيح الفعل والفاعل في الجمل الفعلية المعطاة لك، ثم أقرأ الجمل بنطق صحيح.

1. "شرقت شمس العيد فبادرت السعادة بالجميع"
   - الفعل: شرقت
   - الفاعل: شمس
   - الجملة الفعلية: شرقت شمس

2. "ظهرت مظاهر العيد على كل الوجوه"
   - الفعل: ظهرت
   - الفاعل: العيد
   - الجملة الفعلية: ظهرت مظاهر العيد

3. "في كل البيوت والشوارع تبتسم الأفواه وتردد الألسن العبارات التهنئة"
   - الفعل: تبتسم
   - الفاعل: الأفواه
   - الجملة الفعلية: تبتسم الأفواه

4. "ارتدى الصغير والكبير الملابس الجديدة"
   - الفعل: ارتدى
   - الفاعل: الصغير والكبير
   - الجملة الفعلية: ارتدى الصغير والكبير

5. "حصل الأطفال على العيديات"
   - الفعل: حصل
   - الفاعل: الأطفال
   - الجملة الفعلية: حصل الأطفال

أرجو أن يكون هذا مفيدًا لك! إذا كنت تحتاج إلى توضيح أكثر أو تريد مزيدًا من الأمثلة، فلا تتردد في سؤالي.

